In [1]:
# Bee Hive data https://drive.google.com/file/d/142IBcs6OyQiJxO7owPfkEBFbkrudnh0g/view?usp=sharing

In [2]:
APP = 'BeeHive'

In [3]:
# Install a pip package in the current Jupyter kernel
!{sys.executable} -m pip install -e '../../../Wielder/'
!{sys.executable} -m pip install -e '../'

Obtaining file:///Users/daniel/dev/duds/Wielder
  Preparing metadata (setup.py) ... done
  Attempting uninstall: wielder
    Found existing installation: wielder 0.2.9
    Uninstalling wielder-0.2.9:
      Successfully uninstalled wielder-0.2.9
  Running setup.py develop for wielder

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python -m pip install --upgrade pip
Obtaining file:///Users/daniel/dev/duds/pep-data/src
  Preparing metadata (setup.py) ... done
  Attempting uninstall: pep-data
    Found existing installation: pep-data 0.1
    Uninstalling pep-data-0.1:
      Successfully uninstalled pep-data-0.1
  Running setup.py develop for pep-data

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql.functions import split, when, lit, row_number, udf, col

from pyspark.sql.window import Window

from pep_data.project import get_project_conf

from pep_data.spark.util import field_to_struct

import random

from treelib import Node, Tree

In [5]:
# Create Spark session
spark = SparkSession.builder.appName(APP).getOrCreate()

22/10/25 11:31:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
# Get app configuration from project.conf file
conf = get_project_conf()

In [7]:
#Create schema for the data
cols_name = conf[APP]['cols_name']
cols_double = conf[APP]['cols_double']
cols_integer = conf[APP]['cols_integer']

# Create all the fields
fields = [field_to_struct(header, doubles=cols_double, integers=cols_integer) for header in cols_name]

# Create the schema from th e fields
schema = StructType(fields)

In [8]:
# TODO read from conf
# Read the data from the csv using the schema
df = spark.read.schema(schema).csv('/Users/daniel/dev/duds/pep-data/src/jupyter/BeeHiveTestData.csv')
df.show()

+------+------+--------------------+------------------------+-----------------+-----------+-----------+---------+-----------------+----------------+----------------+
|remove|Bee ID|             remove1|DaughtersEfficiencyScore|          remove2|Father SIZE|Father TYPE|  remove3|                X|               Y|               Z|
+------+------+--------------------+------------------------+-----------------+-----------+-----------+---------+-----------------+----------------+----------------+
|Bee ID|   0_0|DaughtersEfficien...|      -0.164749489890085|Father SIZE&TYPE:|          5|     107027|Locus XYZ| 27.9611146732039|25.5400860475001|24.7985869211487|
|Bee ID|   0_9|DaughtersEfficien...|     -0.0911178061843766|Father SIZE&TYPE:|          8|      35473|Locus XYZ| 3.61617713007262|15.3999367838593|14.1115068303594|
|Bee ID|  0_16|DaughtersEfficien...|      -0.278373748716953|Father SIZE&TYPE:|          9|      72732|Locus XYZ| 6.48713247265339|27.9611146732039|23.5040555402672|
|Bee

In [9]:
# Remove unnecessary columns(the columns with the word remove in them)
cols_to_keep = [x for x in df.columns if 'remove' not in x]
df = df.select(*cols_to_keep)

df.show()

+------+------------------------+-----------+-----------+-----------------+----------------+----------------+
|Bee ID|DaughtersEfficiencyScore|Father SIZE|Father TYPE|                X|               Y|               Z|
+------+------------------------+-----------+-----------+-----------------+----------------+----------------+
|   0_0|      -0.164749489890085|          5|     107027| 27.9611146732039|25.5400860475001|24.7985869211487|
|   0_9|     -0.0911178061843766|          8|      35473| 3.61617713007262|15.3999367838593|14.1115068303594|
|  0_16|      -0.278373748716953|          9|      72732| 6.48713247265339|27.9611146732039|23.5040555402672|
|  0_76|      0.0198831506878141|          6|      49069| 8.28517690605433|21.8811144705459| 10.805611548699|
|  0_35|     -0.0758420403016315|          5|        187|             -1.0|            13.0|            17.0|
|  0_17|      -0.336211045714101|          5|      74276|             -0.6|            12.0|            16.0|
|  0_49|  

In [10]:
# Split Bee_ID column to Cycle(Bee value) and Cycle ID(ID value) columns
df_cleaned = df.withColumn('Cycle', split(col('Bee ID'), '_')\
                           .getItem(0))\
               .withColumn('Cycle ID', split(col('Bee ID'), '_')\
                           .getItem(1))

# Change the type of value in Cycle column from string to integer
df_cleaned= df_cleaned.withColumn("Cycle",col("Cycle")\
                                  .cast(IntegerType()))

df_cleaned.show()

+------+------------------------+-----------+-----------+-----------------+----------------+----------------+-----+--------+
|Bee ID|DaughtersEfficiencyScore|Father SIZE|Father TYPE|                X|               Y|               Z|Cycle|Cycle ID|
+------+------------------------+-----------+-----------+-----------------+----------------+----------------+-----+--------+
|   0_0|      -0.164749489890085|          5|     107027| 27.9611146732039|25.5400860475001|24.7985869211487|    0|       0|
|   0_9|     -0.0911178061843766|          8|      35473| 3.61617713007262|15.3999367838593|14.1115068303594|    0|       9|
|  0_16|      -0.278373748716953|          9|      72732| 6.48713247265339|27.9611146732039|23.5040555402672|    0|      16|
|  0_76|      0.0198831506878141|          6|      49069| 8.28517690605433|21.8811144705459| 10.805611548699|    0|      76|
|  0_35|     -0.0758420403016315|          5|        187|             -1.0|            13.0|            17.0|    0|      35|


In [11]:
# Sort the data frame by Cycle column and add row number for each row(new column with the name Continuous ID)
w = Window().orderBy('Cycle')
df_cleaned = df_cleaned.withColumn('Continuous ID', row_number()\
                                   .over(w))

df_cleaned.show()

22/10/25 11:31:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------+------------------------+-----------+-----------+-----------------+----------------+----------------+-----+--------+-------------+
|Bee ID|DaughtersEfficiencyScore|Father SIZE|Father TYPE|                X|               Y|               Z|Cycle|Cycle ID|Continuous ID|
+------+------------------------+-----------+-----------+-----------------+----------------+----------------+-----+--------+-------------+
|   0_0|      -0.164749489890085|          5|     107027| 27.9611146732039|25.5400860475001|24.7985869211487|    0|       0|            1|
|   0_9|     -0.0911178061843766|          8|      35473| 3.61617713007262|15.3999367838593|14.1115068303594|    0|       9|            2|
|  0_16|      -0.278373748716953|          9|      72732| 6.48713247265339|27.9611146732039|23.5040555402672|    0|      16|            3|
|  0_76|      0.0198831506878141|          6|      49069| 8.28517690605433|21.8811144705459| 10.805611548699|    0|      76|            4|
|  0_35|     -0.07584204030

In [12]:
# Create a dictionary with key = cycle , value = minimun value of Continuous ID of cycle(key)
continuous_min_id_per_cycle = {key : value for key, value  in df_cleaned.groupBy('Cycle').min('Continuous ID').collect()}

continuous_min_id_per_cycle

22/10/25 11:31:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


{0: 1,
 1: 551,
 2: 1859,
 3: 3224,
 4: 4453,
 5: 5171,
 6: 5840,
 7: 6541,
 8: 7394,
 9: 8071,
 10: 8782,
 11: 9290,
 12: 10863,
 13: 12233,
 14: 13375,
 15: 14053,
 16: 14766,
 17: 15458,
 18: 16203,
 19: 16915,
 20: 17700,
 21: 18323,
 22: 19045,
 23: 19859,
 24: 20535,
 25: 21226,
 26: 21862,
 27: 22595,
 28: 23384,
 29: 24147,
 30: 24944,
 31: 25730,
 32: 26403,
 33: 27153,
 34: 27796,
 35: 28645,
 36: 29505,
 37: 30313,
 38: 31053,
 39: 31688,
 40: 32331,
 41: 33170,
 42: 34095,
 43: 34753,
 44: 35502,
 45: 36433,
 46: 37375,
 47: 38147,
 48: 38878,
 49: 39590,
 50: 40388,
 51: 41283,
 52: 43136,
 53: 44282,
 54: 45148,
 55: 46070,
 56: 46859,
 57: 47902,
 58: 48614,
 59: 49444,
 60: 50387,
 61: 51066,
 62: 51920,
 63: 52674,
 64: 53467,
 65: 54124,
 66: 54863,
 67: 55537,
 68: 56387,
 69: 57250,
 70: 58143,
 71: 58892,
 72: 59758,
 73: 60472,
 74: 61223,
 75: 61953,
 76: 62685,
 77: 63550,
 78: 64324,
 79: 65146,
 80: 65767,
 81: 66350,
 82: 67220,
 83: 67951,
 84: 68655,
 85: 6

In [13]:
# Create sorted list of all distinct values of Cycle column
cycles = sorted([i[0] for i in df_cleaned.select('Cycle').distinct().collect()])

cycles

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99]

In [14]:
# Return parent continuous id according to cycle and n
def assert_parent_bee_id(cycle):
    n = 3

    # Get index  of cycle in cycles list
    cycle_index = cycles.index(cycle)

    # Return parent bee id if cycle is 0
    if  not cycle_index :
        return None

    min_cycle_index = 0

    # Update min_cycle_index according to cycle_index and n
    if cycle_index > n:
        min_cycle_index = cycle_index - n

    # Calculate the minimun value for random parent continuous id
    min_rand_value = continuous_min_id_per_cycle[cycles[min_cycle_index]]

    # Calculate the maximum value for random parent continuous id
    max_rand_vale = continuous_min_id_per_cycle[cycles[cycle_index]] - 1

    # Get random value of parent_continuous_id (from min_rand_value to max_rand_vale)
    parent_continuous_id = random.randint(min_rand_value, max_rand_vale)

    return parent_continuous_id

In [26]:
# Convert assert_parent_bee_id(cycle) to user defined function
assert_parent_bee_id_udf = udf(lambda z: assert_parent_bee_id(z))

# Create new column Parent Continuous ID using the assert_parent_bee_id_udf function and Cycle column
# cache() caches the specified data frame in the memory of your cluster's workers
# If executing multiple actions on the same data frame then cache it
df_cleaned = df_cleaned.withColumn("Parent Continuous ID", assert_parent_bee_id_udf(col('Cycle')))\
                        .cache()

df_cleaned.show()

+------+------------------------+-----------+-----------+-----------------+----------------+----------------+-----+--------+-------------+--------------------+
|Bee ID|DaughtersEfficiencyScore|Father SIZE|Father TYPE|                X|               Y|               Z|Cycle|Cycle ID|Continuous ID|Parent Continuous ID|
+------+------------------------+-----------+-----------+-----------------+----------------+----------------+-----+--------+-------------+--------------------+
|   0_0|      -0.164749489890085|          5|     107027| 27.9611146732039|25.5400860475001|24.7985869211487|    0|       0|            1|                null|
|   0_9|     -0.0911178061843766|          8|      35473| 3.61617713007262|15.3999367838593|14.1115068303594|    0|       9|            2|                null|
|  0_16|      -0.278373748716953|          9|      72732| 6.48713247265339|27.9611146732039|23.5040555402672|    0|      16|            3|                null|
|  0_76|      0.0198831506878141|       

In [27]:
# Create a tree using the idead of bfs algorithm

# Get the root parent (first node of the tree)
root_parent_continuous_id = 1
root_parent = df_cleaned.filter(df_cleaned['Continuous ID'] == root_parent_continuous_id).collect()[0]

# list of rows from df_cleand that will be nodes in the tree
tree_nodes = [root]

# Create tree
tree = Tree()

# Add first node(root) to tree
tree.create_node(root['Bee ID'], root['Bee ID'])

# While tree_nodes not empty pop first value (parent) serach for its kids append them to tree_nodes and add them to the tree
while tree_nodes:
    # Get first value in tree_nodes
    parent = tree_nodes.pop()

    #find all rows in df_cleaned which their value in
    kids = df_cleaned.filter(col('Parent Continuous ID') == parent['Continuous ID'])

    for k in kids.collect():
        tree_nodes.append(k)
        tree.create_node(k['Bee ID'], k['Bee ID'], parent['Bee ID'] )


tree.show()

0_0
├── 1_1037
│   ├── 2_1995
│   │   └── 4_2067
│   │       └── 7_1581
│   └── 3_2120
│       └── 5_1199
│           └── 6_2165
├── 1_1820
│   ├── 2_1150
│   ├── 2_1337
│   │   └── 3_1302
│   │       ├── 6_1874
│   │       │   └── 7_2230
│   │       └── 6_570
│   │           ├── 7_14
│   │           │   ├── 8_904
│   │           │   │   ├── 10_1126
│   │           │   │   │   ├── 12_1438
│   │           │   │   │   │   ├── 13_416
│   │           │   │   │   │   │   └── 16_1607
│   │           │   │   │   │   │       ├── 17_2717
│   │           │   │   │   │   │       └── 17_572
│   │           │   │   │   │   │           ├── 18_2217
│   │           │   │   │   │   │           │   └── 19_284
│   │           │   │   │   │   │           │       ├── 20_1957
│   │           │   │   │   │   │           │       │   ├── 22_2791
│   │           │   │   │   │   │           │       │   │   ├── 23_2293
│   │           │   │   │   │   │           │       │   │   │   └── 24_7
│   │           │   │ 

In [ ]:
# from operator import add
def test2(a,b):
    return a+b

spark.sparkContext.parallelize([1, 2, 3, 4, 5]).fold(1, test2)

In [ ]:
def test1(tree_dict, row):

    # for r in row:
    #     if r[10] == 1:
    #         tree_dict.append(row)

    # for column in tree_dict:
    #     if column['Continuous ID'] == row[0]:
    #         tree_dict.append(row)

    # for column in tree_dict:
    #     #tree_dict[column].append(row)
    #     for r in list(row):
    #         print(r)
    #     print("-------")

    # for column in tree_dict:
    #     print(type(column))
    #     # if column['Continuous ID'] == list(row.keys())[0][10]:
    #     #     print(row)
    # print('---------')

    # print(type(row))
    # print(row[10])
    # #print((list(row))[0][10])

    return tree_dict

root = df_cleaned.head()
df_cleaned.rdd.fold([],test1 )

In [ ]:
d = {1:['a'], 2: ['b']}
d[1].append('c')
print(d)